# This code trains the ciraf10 model, which can identify images based on the dataset

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks, optimizers


# Reproducibility

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# GPU setup (use GPU if available)

In [ ]:
try:
	gpus = tf.config.list_physical_devices('GPU')
	if gpus:
		for gpu in gpus:
			try:
				tf.config.experimental.set_memory_growth(gpu, True)
			except Exception:
				pass
		print(f"GPUs detected: {len(gpus)} (using GPU acceleration)")
		# Optional: enable mixed precision for speed on modern GPUs
		try:
			from tensorflow.keras import mixed_precision
			mixed_precision.set_global_policy('mixed_float16')
		except Exception:
			pass
	else:
		print("No GPU detected. Training will use CPU.")
except Exception as e:
	print(f"GPU setup warning: {e}")

GPUs detected: 1 (using GPU acceleration)


# Paths and output setup

In [ ]:
PROJECT_ROOT = os.path.abspath('.')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'output')
PLOTS_DIR = os.path.join(OUTPUT_DIR, 'plots')
LOGS_DIR = os.path.join(OUTPUT_DIR, 'logs')
MODEL_NAME = 'My-App-Model.h5'
BEST_MODEL_PATH = os.path.join(OUTPUT_DIR, MODEL_NAME)
CONF_MATRIX_PNG = os.path.join(PLOTS_DIR, 'confusion_matrix.png')
CSV_LOG_PATH = os.path.join(LOGS_DIR, 'training_log.csv')

for d in [OUTPUT_DIR, PLOTS_DIR, LOGS_DIR]:
	os.makedirs(d, exist_ok=True)


# Data loading and preprocessing

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

class_names = [
	'airplane', 'automobile', 'bird', 'cat', 'deer',
	'dog', 'frog', 'horse', 'ship', 'truck'
]


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


# Model definition (CNN)

In [ ]:
def build_model(input_shape=(32, 32, 3), num_classes=10, use_mixed_precision=False):
	inputs = layers.Input(shape=input_shape)

	# In-graph data augmentation
	x = layers.RandomFlip('horizontal')(inputs)
	x = layers.RandomRotation(0.1)(x)
	x = layers.RandomZoom(0.1)(x)

	# Conv blocks
	def conv_block(x, filters):
		x = layers.Conv2D(filters, 3, padding='same')(x)
		x = layers.BatchNormalization()(x)
		x = layers.ReLU()(x)
		x = layers.Conv2D(filters, 3, padding='same')(x)
		x = layers.BatchNormalization()(x)
		x = layers.ReLU()(x)
		x = layers.MaxPooling2D()(x)
		x = layers.Dropout(0.25)(x)
		return x

	x = conv_block(x, 64)
	x = conv_block(x, 128)
	x = conv_block(x, 256)

	x = layers.GlobalAveragePooling2D()(x)
	x = layers.Dropout(0.4)(x)
	x = layers.Dense(256)(x)
	x = layers.ReLU()(x)
	x = layers.Dropout(0.3)(x)

	# If mixed precision is enabled, use float32 for final layer for numerical stability
	final_dtype = 'float32'
	outputs = layers.Dense(num_classes, activation='softmax', dtype=final_dtype)(x)

	model = models.Model(inputs, outputs, name='MyAppCIFAR10')
	return model

use_mixed = False
try:
	from tensorflow.keras import mixed_precision
	use_mixed = (mixed_precision.global_policy().compute_dtype == 'float16')
except Exception:
	use_mixed = False

model = build_model(use_mixed_precision=use_mixed)


Optimizer and compile

In [ ]:
base_lr = 1e-3
opt = optimizers.Adam(learning_rate=base_lr)
model.compile(
	optimizer=opt,
	loss='categorical_crossentropy',
	metrics=['accuracy']
)

model.summary()


Model: "MyAppCIFAR10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 32, 32, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │             

 Total params: 1,217,354 (4.64 MB)

 Trainable params: 1,215,562 (4.64 MB)

 Non-trainable params: 1,792 (7.00 KB)

# Callbacks

In [ ]:
cb = [
	callbacks.ModelCheckpoint(
		BEST_MODEL_PATH,
		monitor='val_accuracy',
		save_best_only=True,
		save_weights_only=False,
		mode='max',
		verbose=1
	),
	callbacks.ReduceLROnPlateau(
		monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
	),
	callbacks.EarlyStopping(
		monitor='val_accuracy', patience=12, restore_best_weights=True, verbose=1
	),
	callbacks.CSVLogger(CSV_LOG_PATH)
]


# Training

In [ ]:
epochs = 50
batch_size = 128

history = model.fit(
        x_train, y_train_cat,
        validation_split=0.1,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=cb,
        verbose=1
)

# Ensure best model is saved
# Change the model saving format to the native Keras format
BEST_MODEL_PATH = os.path.join(OUTPUT_DIR, MODEL_NAME.replace('.h5', '.keras'))
model.save(BEST_MODEL_PATH)
print(f"Saved model to: {BEST_MODEL_PATH}")

Epoch 1/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.2931 - loss: 1.9103
Epoch 1: val_accuracy improved from -inf to 0.18600, saving model to /content/output/My-App-Model.h5


352/352 ━━━━━━━━━━━━━━━━━━━━ 39s 64ms/step - accuracy: 0.2933 - loss: 1.9098 - val_accuracy: 0.1860 - val_loss: 2.6521 - learning_rate: 0.0010
Epoch 2/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4844 - loss: 1.4088
Epoch 2: val_accuracy improved from 0.18600 to 0.51420, saving model to /content/output/My-App-Model.h5


352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.4844 - loss: 1.4086 - val_accuracy: 0.5142 - val_loss: 1.3093 - learning_rate: 0.0010
Epoch 3/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5511 - loss: 1.2487
Epoch 3: val_accuracy did not improve from 0.51420
352/352 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.5512 - loss: 1.2486 - val_accuracy: 0.4922 - val_loss: 1.4101 - learning_rate: 0.0010
Epoch 4/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5965 - loss: 1.1339
Epoch 4: val_accuracy did not improve from 0.51420
352/352 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - accuracy: 0.5965 - loss: 1.1338 - val_accuracy: 0.5042 - val_loss: 1.7020 - learning_rate: 0.0010
Epoch 5/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6305 - loss: 1.0389
Epoch 5: val_accuracy improved from 0.51420 to 0.61160, saving model to /content/output/My-App-Model.h5


352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.6305 - loss: 1.0389 - val_accuracy: 0.6116 - val_loss: 1.2297 - learning_rate: 0.0010
Epoch 6/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6561 - loss: 0.9709
Epoch 6: val_accuracy improved from 0.61160 to 0.66380, saving model to /content/output/My-App-Model.h5


352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.6561 - loss: 0.9709 - val_accuracy: 0.6638 - val_loss: 1.0063 - learning_rate: 0.0010
Epoch 7/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.6815 - loss: 0.9101
Epoch 7: val_accuracy improved from 0.66380 to 0.68300, saving model to /content/output/My-App-Model.h5


...


Epoch 41: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.8753 - loss: 0.3569 - val_accuracy: 0.8726 - val_loss: 0.3861 - learning_rate: 2.5000e-04
Epoch 42/50
351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8829 - loss: 0.3416
Epoch 42: val_accuracy did not improve from 0.87260
352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.8829 - loss: 0.3416 - val_accuracy: 0.8706 - val_loss: 0.3938 - learning_rate: 1.2500e-04
Epoch 43/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8824 - loss: 0.3415
Epoch 43: val_accuracy improved from 0.87260 to 0.87400, saving model to /content/output/My-App-Model.h5


352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.8824 - loss: 0.3415 - val_accuracy: 0.8740 - val_loss: 0.3964 - learning_rate: 1.2500e-04
Epoch 44/50
351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8820 - loss: 0.3397
Epoch 44: val_accuracy did not improve from 0.87400
352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.8820 - loss: 0.3397 - val_accuracy: 0.8738 - val_loss: 0.3818 - learning_rate: 1.2500e-04
Epoch 45/50
351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8852 - loss: 0.3373
Epoch 45: val_accuracy improved from 0.87400 to 0.87860, saving model to /content/output/My-App-Model.h5


352/352 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.8852 - loss: 0.3373 - val_accuracy: 0.8786 - val_loss: 0.3666 - learning_rate: 1.2500e-04
Epoch 46/50
351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8869 - loss: 0.3328
Epoch 46: val_accuracy did not improve from 0.87860
352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.8869 - loss: 0.3328 - val_accuracy: 0.8704 - val_loss: 0.3959 - learning_rate: 1.2500e-04
Epoch 47/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8825 - loss: 0.3407
Epoch 47: val_accuracy did not improve from 0.87860
352/352 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.8825 - loss: 0.3407 - val_accuracy: 0.8746 - val_loss: 0.3713 - learning_rate: 1.2500e-04
Epoch 48/50
351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8877 - loss: 0.3322
Epoch 48: val_accuracy did not improve from 0.87860
352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 45ms/step - accuracy: 0.8877 - loss: 0.3321 - val_accuracy: 0.8774 - val_loss: 0.3807 - learning_rate: 1.2

352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.8889 - loss: 0.3237 - val_accuracy: 0.8804 - val_loss: 0.3568 - learning_rate: 1.2500e-04
Restoring model weights from the end of the best epoch: 50.


Saved model to: /content/output/My-App-Model.h5


# Evaluation

In [ ]:
print("Evaluating on test set...")
loss, acc = model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test accuracy: {acc:.4f}")

# Predictions and reports
y_pred_probs = model.predict(x_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test.flatten()

report = classification_report(y_true, y_pred, target_names=class_names)
print("\nClassification Report:\n", report)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('CIFAR-10 Confusion Matrix - My-App-Model')
plt.tight_layout()
plt.savefig(CONF_MATRIX_PNG)
print(f"Confusion matrix plot saved to: {CONF_MATRIX_PNG}")
plt.close()


Evaluating on test set...
Test accuracy: 0.8781

Classification Report:
               precision    recall  f1-score   support

    airplane       0.91      0.87      0.89      1000
  automobile       0.91      0.95      0.93      1000
        bird       0.88      0.83      0.85      1000
         cat       0.82      0.74      0.78      1000
        deer       0.86      0.87      0.86      1000
         dog       0.87      0.78      0.83      1000
        frog       0.85      0.94      0.89      1000
       horse       0.88      0.93      0.90      1000
        ship       0.91      0.94      0.92      1000
       truck       0.87      0.94      0.90      1000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

Confusion matrix plot saved to: /content/output/plots/confusion_matrix.png


# Export label mapping for inference

In [ ]:
with open(os.path.join(OUTPUT_DIR, 'class_names.txt'), 'w') as f:
	for name in class_names:
		f.write(name + '\n')
print(f"Class names saved to: {os.path.join(OUTPUT_DIR, 'class_names.txt')}")


Class names saved to: /content/output/class_names.txt
